In [1]:
# Proyecto ULiège - Detección y Conteo de Fauna con YOLOv8m
# Pipeline completo: entrenamiento, evaluación y conversión a centroides (HerdNet)

In [2]:
import os, json, random, hashlib
from pathlib import Path
from collections import defaultdict
import numpy as np
from PIL import Image, ImageDraw
import torch
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import yaml
import matplotlib.pyplot as plt
from tqdm import tqdm
import imagehash

# Reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [3]:
# 1. Configuración de rutas y entorno
DATASET_DIR = Path("./dataset").resolve()
GROUNDTRUTH = DATASET_DIR / "groundtruth" / "json" / "big_size"
TRAIN_JSON = GROUNDTRUTH / "train_big_size_A_B_E_K_WH_WB.json"
VAL_JSON = GROUNDTRUTH / "val_big_size_A_B_E_K_WH_WB.json"
TEST_JSON = GROUNDTRUTH / "test_big_size_A_B_E_K_WH_WB.json"
TRAIN_IMGS = DATASET_DIR / "train"
VAL_IMGS = DATASET_DIR / "val"
TEST_IMGS = DATASET_DIR / "test"
TRAIN_SUB = DATASET_DIR / "train_subframes"
EXPERIMENTS_DIR = (DATASET_DIR.parent / "experimentos_yolo_final").resolve()
EXPERIMENTS_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
# 2. Verificación anti-data leakage
def check_overlap(json1, json2):
    with open(json1) as f1, open(json2) as f2:
        set1 = {img["file_name"] for img in json.load(f1)["images"]}
        set2 = {img["file_name"] for img in json.load(f2)["images"]}
        overlap = set1.intersection(set2)
        print(f"Solapamiento entre {json1.name} y {json2.name}: {len(overlap)} imágenes")
        return overlap

def check_hash_duplicates(dir_a, dir_b, sample=300):
    """Compara hashes perceptuales entre dos carpetas (para detectar subframes duplicados)"""
    imgs_a = list(dir_a.glob("*.jpg"))[:sample]
    imgs_b = list(dir_b.glob("*.jpg"))[:sample]
    hashes_a = {imagehash.phash(Image.open(p)) for p in imgs_a}
    dup = 0
    for pb in imgs_b:
        hb = imagehash.phash(Image.open(pb))
        if any(abs(hb - ha) < 5 for ha in hashes_a):
            dup += 1
    print(f"Posibles duplicados perceptuales entre {dir_a.name} y {dir_b.name}: {dup}")
    return dup

# Ejecutar verificaciones
check_overlap(TRAIN_JSON, VAL_JSON)
check_overlap(TRAIN_JSON, TEST_JSON)
check_overlap(VAL_JSON, TEST_JSON)
check_hash_duplicates(TRAIN_SUB, VAL_IMGS, sample=200)

Solapamiento entre train_big_size_A_B_E_K_WH_WB.json y val_big_size_A_B_E_K_WH_WB.json: 0 imágenes
Solapamiento entre train_big_size_A_B_E_K_WH_WB.json y test_big_size_A_B_E_K_WH_WB.json: 0 imágenes
Solapamiento entre val_big_size_A_B_E_K_WH_WB.json y test_big_size_A_B_E_K_WH_WB.json: 0 imágenes
Posibles duplicados perceptuales entre train_subframes y val: 0


0

In [5]:
# 3. Conversión de COCO JSON a YOLO TXT

# Rutas de salida para las etiquetas YOLO
YOLO_LABELS_DIR = DATASET_DIR / "labels"
YOLO_LABELS_TRAIN = YOLO_LABELS_DIR / "train"
YOLO_LABELS_VAL = YOLO_LABELS_DIR / "val"
YOLO_LABELS_TEST = YOLO_LABELS_DIR / "test"

YOLO_LABELS_TRAIN.mkdir(parents=True, exist_ok=True)
YOLO_LABELS_VAL.mkdir(parents=True, exist_ok=True)
YOLO_LABELS_TEST.mkdir(parents=True, exist_ok=True)

def convert_coco_to_yolo(json_path, output_dir):
    """Convierte un archivo COCO JSON a archivos de etiquetas YOLO TXT."""
    print(f"Iniciando conversión de: {json_path}")
    coco = COCO(json_path)
    
    # Mapeo de IDs de imágenes a sus nombres de archivo y dimensiones
    img_details = {img['id']: (img['file_name'], img['width'], img['height']) for img in coco.dataset['images']}
    
    # Obtener todas las anotaciones
    annotations = coco.anns.values()
    
    # Inicializar un diccionario para acumular anotaciones por imagen
    yolo_annotations = defaultdict(list)
    
    for ann in tqdm(annotations, desc=f"Convirtiendo {json_path.name}"):
        image_id = ann['image_id']
        category_id = ann['category_id']
        bbox = ann['bbox']
        
        # Obtener detalles de la imagen
        try:
            file_name, img_w, img_h = img_details[image_id]
        except KeyError:
            continue
        
        # 1. Desnormalizar las coordenadas
        x_min, y_min, w, h = bbox
        
        # 2. Calcular el centro y el tamaño normalizado (formato YOLO)
        x_center = (x_min + w / 2) / img_w
        y_center = (y_min + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h
        
        # 3. Mapear el ID de categoría (si es necesario)
        # category_ids en el JSON corresponden
        # al índice de clase (0 a 5) que se usa en el YAML.
        # Si no es así, mapearlos aquí.
        # Para simplificar y alineado a las 6 clases:
        # 1 -> 0, 2 -> 1, ..., 6 -> 5
        class_id_yolo = category_id - 1
        
        yolo_line = f"{class_id_yolo} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
        
        # El nombre del archivo de etiqueta es el mismo que el nombre de la imagen
        # pero con extensión .txt
        label_file_name = Path(file_name).stem + ".txt"
        yolo_annotations[label_file_name].append(yolo_line)
    
    # Escribir los archivos de anotación TXT
    for file_name, lines in yolo_annotations.items():
        with open(output_dir / file_name, 'w') as f:
            f.write("\n".join(lines))
    
    print(f"Conversión completada. Etiquetas guardadas en: {output_dir}")

# Ejecutar la conversión para los conjuntos de Train, Val y Test
convert_coco_to_yolo(TRAIN_JSON, YOLO_LABELS_TRAIN)
convert_coco_to_yolo(VAL_JSON, YOLO_LABELS_VAL)
convert_coco_to_yolo(TEST_JSON, YOLO_LABELS_TEST)

# 3.1: Ajuste Físico de la Estructura de Carpetas para YOLOv8

print("Ajustando estructura de etiquetas para el motor de YOLOv8...")

import shutil

def reorganize_dataset_for_yolo(base_dir, labels_source):
    """
    Reorganiza el dataset para que siga la estructura esperada por YOLOv8:
    base_dir/
        images/  <- mueve aquí las imágenes que están directamente en base_dir
        labels/  <- mueve aquí las etiquetas desde labels_source
    """
    images_dir = base_dir / "images"
    labels_dir = base_dir / "labels"
    
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)
    
    # Mover imágenes de base_dir/*.jpg a base_dir/images/
    moved_images = 0
    for img_file in base_dir.glob("*.jpg"):
        try:
            shutil.move(str(img_file), str(images_dir / img_file.name))
            moved_images += 1
        except Exception as e:
            print(f"Error moviendo imagen {img_file.name}: {e}")
    
    # Intentar también con extensión .JPG (mayúscula)
    for img_file in base_dir.glob("*.JPG"):
        try:
            shutil.move(str(img_file), str(images_dir / img_file.name))
            moved_images += 1
        except Exception as e:
            print(f"Error moviendo imagen {img_file.name}: {e}")
    
    print(f"Movidas {moved_images} imágenes a {images_dir}")
    
    # Mover etiquetas desde labels_source a base_dir/labels/
    moved_labels = 0
    for label_file in labels_source.glob("*.txt"):
        try:
            shutil.move(str(label_file), str(labels_dir / label_file.name))
            moved_labels += 1
        except Exception as e:
            print(f"Error moviendo etiqueta {label_file.name}: {e}")
    
    print(f"Movidas {moved_labels} etiquetas a {labels_dir}")
    
    return moved_images, moved_labels

# Reorganizar train
print("\nReorganizando conjunto TRAIN...")
reorganize_dataset_for_yolo(TRAIN_IMGS, YOLO_LABELS_TRAIN)

# Reorganizar val
print("\nReorganizando conjunto VAL...")
reorganize_dataset_for_yolo(VAL_IMGS, YOLO_LABELS_VAL)

# Reorganizar test
print("\nReorganizando conjunto TEST...")
reorganize_dataset_for_yolo(TEST_IMGS, YOLO_LABELS_TEST)

print("\nAjuste completado. La estructura de carpetas ahora es la esperada por YOLOv8.")

Iniciando conversión de: C:\Users\durle\anaconda3\Fauna_Detection\dataset\groundtruth\json\big_size\train_big_size_A_B_E_K_WH_WB.json
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


Convirtiendo train_big_size_A_B_E_K_WH_WB.json: 100%|██████████| 6962/6962 [00:00<00:00, 347913.70it/s]


Conversión completada. Etiquetas guardadas en: C:\Users\durle\anaconda3\Fauna_Detection\dataset\labels\train
Iniciando conversión de: C:\Users\durle\anaconda3\Fauna_Detection\dataset\groundtruth\json\big_size\val_big_size_A_B_E_K_WH_WB.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Convirtiendo val_big_size_A_B_E_K_WH_WB.json: 100%|██████████| 978/978 [00:00<00:00, 325248.12it/s]


Conversión completada. Etiquetas guardadas en: C:\Users\durle\anaconda3\Fauna_Detection\dataset\labels\val
Iniciando conversión de: C:\Users\durle\anaconda3\Fauna_Detection\dataset\groundtruth\json\big_size\test_big_size_A_B_E_K_WH_WB.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Convirtiendo test_big_size_A_B_E_K_WH_WB.json: 100%|██████████| 2299/2299 [00:00<00:00, 176865.46it/s]


Conversión completada. Etiquetas guardadas en: C:\Users\durle\anaconda3\Fauna_Detection\dataset\labels\test
Ajustando estructura de etiquetas para el motor de YOLOv8...

Reorganizando conjunto TRAIN...
Movidas 928 imágenes a C:\Users\durle\anaconda3\Fauna_Detection\dataset\train\images
Movidas 928 etiquetas a C:\Users\durle\anaconda3\Fauna_Detection\dataset\train\labels

Reorganizando conjunto VAL...
Movidas 111 imágenes a C:\Users\durle\anaconda3\Fauna_Detection\dataset\val\images
Movidas 111 etiquetas a C:\Users\durle\anaconda3\Fauna_Detection\dataset\val\labels

Reorganizando conjunto TEST...
Movidas 258 imágenes a C:\Users\durle\anaconda3\Fauna_Detection\dataset\test\images
Movidas 258 etiquetas a C:\Users\durle\anaconda3\Fauna_Detection\dataset\test\labels

Ajuste completado. La estructura de carpetas ahora es la esperada por YOLOv8.


In [6]:
# 4. Archivo data.yaml (dataset completo)

import platform
import sys

try:
    DATASET_DIR = Path("./dataset").resolve()
except Exception:
    DATASET_DIR = Path.cwd() / "dataset"

# Actualizar rutas para apuntar a las carpetas correctas
TRAIN_IMGS_YOLO = DATASET_DIR / "train" / "images"
VAL_IMGS_YOLO = DATASET_DIR / "val" / "images"
TEST_IMGS_YOLO = DATASET_DIR / "test" / "images"

yaml_path = EXPERIMENTS_DIR / "data_full.yaml"

data_yaml = {
    "path": str(DATASET_DIR),
    "train": str(TRAIN_IMGS_YOLO.relative_to(DATASET_DIR)),
    "val": str(VAL_IMGS_YOLO.relative_to(DATASET_DIR)),
    "nc": 6,
    "names": ["class_0", "class_1", "class_2", "class_3", "class_4", "class_5"]
}

with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

print(f"Experiments folder creado en: {EXPERIMENTS_DIR}")
print(f"data_full.yaml guardado en: {yaml_path}")
print(f"\nContenido escrito en data_full.yaml:")
print("-" * 40)
with open(yaml_path) as f:
    print(f.read())

Experiments folder creado en: C:\Users\durle\anaconda3\Fauna_Detection\experimentos_yolo_final
data_full.yaml guardado en: C:\Users\durle\anaconda3\Fauna_Detection\experimentos_yolo_final\data_full.yaml

Contenido escrito en data_full.yaml:
----------------------------------------
path: C:\Users\durle\anaconda3\Fauna_Detection\dataset
train: train\images
val: val\images
nc: 6
names:
- class_0
- class_1
- class_2
- class_3
- class_4
- class_5



In [7]:
# 5. Entrenamiento YOLOv8
model = YOLO("yolov8m.pt")
train_params = {
    "data": str(yaml_path),
    "epochs": 50,
    "imgsz": 1024,
    "batch": 8,
    "project": str(EXPERIMENTS_DIR),
    "name": "ULiege_YOLOv8m_full15",
    "device": 0 if torch.cuda.is_available() else "cpu",
    "workers": 8,
    "patience": 15,
    "save_period": 10,
}

results = model.train(**train_params)
best_weights = Path(results.save_dir) / "weights" / "best.pt"
print("Pesos guardados en:", best_weights)

New https://pypi.org/project/ultralytics/8.3.214 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.207  Python-3.10.18 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080 Laptop GPU, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\durle\anaconda3\Fauna_Detection\experimentos_yolo_final\data_full.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, 

2025/10/14 17:15:09 INFO mlflow.tracking.fluent: Experiment with name 'C:\Users\durle\anaconda3\Fauna_Detection\experimentos_yolo_final' does not exist. Creating a new experiment.


MLflow: logging run_id(7591596c604e455fb114e29cfbf586f3) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to C:\Users\durle\anaconda3\Fauna_Detection\experimentos_yolo_final\ULiege_YOLOv8m_full15
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/50      8.38G      3.054      16.63      1.005         17       1024: 100% ━━━━━━━━━━━━ 116/116 2.5it/s 45.6s0.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 4.9it/s 1.4s0.2s
                   all        111        978      0.476      0.231      0.167     0.0498

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50      8.08G      2.964      3.776     0.9612         86       1024: 100% ━━━

In [8]:
# 6. Evaluación inicial- conjunto de validación: mAP + métricas de conteo (MAE, RMSE)
def yolo_predictions_to_coco(model, image_dir, conf=0.25, iou=0.45):
    preds = []
    for img_path in tqdm(image_dir.glob("*.jpg"), desc="Inferencia val"):
        res = model.predict(source=str(img_path), conf=conf, iou=iou, verbose=False)
        if not res:
            continue
        boxes = res[0].boxes
        
        for b in boxes:
            x1, y1, x2, y2 = b.xyxy[0].cpu().numpy()
            w, h = x2 - x1, y2 - y1
            preds.append({
                "image_id": img_path.name,
                "category_id": int(b.cls[0]),
                "bbox": [float(x1), float(y1), float(w), float(h)],
                "score": float(b.conf[0])
            })
    
    # Intentar también con extensión .JPG (mayúscula)
    for img_path in tqdm(image_dir.glob("*.JPG"), desc="Inferencia val (JPG)"):
        res = model.predict(source=str(img_path), conf=conf, iou=iou, verbose=False)
        if not res:
            continue
        boxes = res[0].boxes
        
        for b in boxes:
            x1, y1, x2, y2 = b.xyxy[0].cpu().numpy()
            w, h = x2 - x1, y2 - y1
            preds.append({
                "image_id": img_path.name,
                "category_id": int(b.cls[0]),
                "bbox": [float(x1), float(y1), float(w), float(h)],
                "score": float(b.conf[0])
            })
    
    return preds

def conteo_mae_rmse(gt_json, preds):
    coco_gt = COCO(gt_json)
    gt_counts = defaultdict(int)
    for a in coco_gt.loadAnns(coco_gt.getAnnIds()):
        gt_counts[coco_gt.loadImgs(a["image_id"])[0]["file_name"]] += 1
    pred_counts = defaultdict(int)
    for p in preds:
        pred_counts[p["image_id"]] += 1
    ids = sorted(set(gt_counts) | set(pred_counts))
    diffs = np.array([pred_counts[i] - gt_counts[i] for i in ids])
    mae = float(np.mean(np.abs(diffs)))
    rmse = float(np.sqrt(np.mean(diffs**2)))
    return mae, rmse

trained_model = YOLO(str(best_weights))
preds = yolo_predictions_to_coco(trained_model, VAL_IMGS_YOLO)
mae, rmse = conteo_mae_rmse(VAL_JSON, preds)
print(f"MAE: {mae:.3f} | RMSE: {rmse:.3f}")

Inferencia val: 111it [00:16,  6.65it/s]
Inferencia val (JPG): 111it [00:15,  7.25it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
MAE: 4.811 | RMSE: 7.315


In [9]:
# 7. Conversión a centroides (para comparación HerdNet)
def centroides_from_bboxes(preds):
    pts = defaultdict(list)
    for p in preds:
        x, y, w, h = p["bbox"]
        pts[p["image_id"]].append((x + w/2, y + h/2))
    return pts

centros_pred = centroides_from_bboxes(preds)
img_sample = list(VAL_IMGS_YOLO.glob(".jpg"))[0] if list(VAL_IMGS_YOLO.glob(".jpg")) else list(VAL_IMGS_YOLO.glob("*.JPG"))[0]
img = Image.open(img_sample).convert("RGB")
draw = ImageDraw.Draw(img)
for (cx, cy) in centros_pred[img_sample.name]:
    r = 5
    draw.ellipse((cx - r, cy - r, cx + r, cy + r), outline="red", width=2)

plt.imshow(img)
plt.axis("off")
plt.title("Predicciones convertidas a centroides (comparables con HerdNet)")
plt.show()

<Figure size 640x480 with 1 Axes>

In [10]:
# 8. Evaluación Final con el Conjunto de Test (Rendimiento Definitivo)

print("\n--- Iniciando Evaluación Final con el Conjunto de Test ---")

# Se reutiliza el 'trained_model' cargado con los mejores pesos.

# 1. Generar predicciones en formato COCO JSON para el conjunto de Test
# Se utiliza la carpeta de imágenes de Test (TEST_IMGS_YOLO)

preds_test = yolo_predictions_to_coco(trained_model, TEST_IMGS_YOLO)

# 2. Calcular métricas de Conteo (MAE y RMSE) usando el JSON de Test
# Se utiliza el JSON de Ground Truth de Test (TEST_JSON)
mae_test, rmse_test = conteo_mae_rmse(TEST_JSON, preds_test)

print(f"\n Resultados de Conteo en el Conjunto de Test (Métricas de HerdNet):")
print(f"MAE (Error Absoluto Medio): {mae_test:.3f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse_test:.3f}")


--- Iniciando Evaluación Final con el Conjunto de Test ---


Inferencia val: 258it [00:43,  5.89it/s]
Inferencia val (JPG): 258it [00:35,  7.20it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

 Resultados de Conteo en el Conjunto de Test (Métricas de HerdNet):
MAE (Error Absoluto Medio): 5.229
RMSE (Raíz del Error Cuadrático Medio): 10.201


In [12]:
# 9. Conversión a Centroides para Test (Comparación HerdNet)

print("--- Generando Salida en Formato Centroides (HerdNet Style) para Test ---")

# 'preds_test' se obtiene de la Celda 8 (Evaluación Final con Test)
# La función 'centroides_from_bboxes' definida previamente
centroides_test = centroides_from_bboxes(preds_test)

# Opcional: Guardar el resultado en un archivo JSON o TXT si es necesario para comparaciones posteriores
#El formato de 'centroides_test' será un diccionario: {file_name: [(x_center, y_center), ...]}

# Guardar si se necesita el archivo:
output_centroid_path = EXPERIMENTS_DIR / "test_centroides.json"
with open(output_centroid_path, 'w') as f:
    # Nota: Es posible que se necesite convertir defaultdict a dict si se usa json.dump
    json.dump(centroides_test, f, indent=4)

print(f" Centroides generados para {len(centroides_test)} imágenes del Test.")
print(f"Resultado guardado en: {output_centroid_path}")

--- Generando Salida en Formato Centroides (HerdNet Style) para Test ---
 Centroides generados para 161 imágenes del Test.
Resultado guardado en: C:\Users\durle\anaconda3\Fauna_Detection\experimentos_yolo_final\test_centroides.json
